In [1]:
%matplotlib inline
import math
import os
import data_util
import BMapModel
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import faiss
import util
import scipy
import tensorflow as tf
from sklearn.externals import joblib # store classifiers

from scipy.sparse import save_npz, load_npz
from numpy.random import normal # generate transforming matrix
from tensorflow.python.framework import function
from joblib import Parallel, delayed # Multitread
from pytictoc import TicToc
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import scipy.io as sio


Failed to load GPU Faiss: No module named swigfaiss_gpu
Faiss falling back to CPU-only.


In [2]:
ls ../data/

AmazonCat/          Bibtex/          Eurlex/     README_Datasets*
AmazonCat-14K/      Delicious/       Mediamill/  Wiki10/
AmazonCat-14K.zip*  DeliciousLarge/  RCV1-x/     XMLDatasetRead/


In [7]:
data_dir = "../data"
model_dir = "../model/model7"
path='/Eurlex'

model_path = model_dir + path
data_path = data_dir + path
tf_board_path = '/tmp/tensorflow/SGH_multilabel' + path
num_core = -1
L_hat_ratio = 0.5 # useful when calculate L_hat = klogn*ratio
L_hat = 100
time = TicToc()
[X_tr, X_te, Y_tr, Y_te] = [load_npz(os.path.join(data_path, '{}.npz'.format(name)))\
                            for name in ['X_tr', 'X_te', 'Y_tr', 'Y_te']]

In [4]:
X_tr.shape, X_te.shape, Y_tr.shape, Y_te.shape

((15539, 5000), (15539, 5000), (15539, 3993), (15539, 3993))

## Step 1: use SGH to learn a embedding for y
y-->z-->y to optimize a embeding $z\in\{0,1\}^{100}$

In [5]:
ymean = Y_tr.mean(axis=0).astype('float64')
yvar = np.clip(Y_tr.toarray().var(axis=0), 1e-7, np.inf).astype('float64')


In [ ]:
import numpy as np

import tensorflow as tf

from tensorflow.python.framework import function
#from tensorflow.train import Saver
# reduced MRF with stochastic neuron 

dim_input = Y_tr.shape[1]
dim_hidden= 100
batch_size = 1500
learning_rate = 1e-2
max_iter = 10000

alpha = 1e-3
beta = 1e-3

def VAE_stoc_neuron(alpha, batch_size, learning_rate, max_iter):
    
    g = tf.Graph()
    dtype = tf.float32
    
    with g.as_default():
        x = tf.placeholder(dtype, [None, dim_input], name='x')
        
        # define doubly stochastic neuron with gradient by DeFun
        #gradient_func: [x1,...,xn,dL/dy1,dL/dy2,...,dL/dym]---->[dL/dx1,...,dL/dxn]
        @function.Defun(dtype, dtype, dtype)
        def DoublySNGrad(logits, epsilon, dprev):
            prob = 1.0 / (1 + tf.exp(-logits))

            # unbiased
            dlogits = prob * (1 - prob) * (dprev)
            return dlogits, 0.

        @function.Defun(dtype, dtype, grad_func=DoublySNGrad)
        def DoublySN(logits, epsilon):
            prob = 1.0 / (1 + tf.exp(-logits))
            yout = (tf.sign(prob - epsilon) + 1.0) / 2.0
            return yout
        
        with tf.name_scope('encode'):
            wencode = tf.Variable(tf.random_normal([dim_input, dim_hidden], stddev=1.0 / tf.sqrt(float(dim_input)), dtype=dtype),
                                       name='wencode')
            bencode = tf.Variable(tf.random_normal([dim_hidden], dtype=dtype), name='bencode')
            hencode = tf.matmul(x, wencode) + bencode
            # determinastic output
            hepsilon = tf.ones(shape=tf.shape(hencode), dtype=dtype, name='hepsilon') * .5
            with tf.name_scope('MLE_logistic_regression'):
                yout = DoublySN(hencode, hepsilon) #activation
            
        with tf.name_scope('decode'):
            with tf.name_scope('scale'):
                scale_para = tf.Variable(tf.constant(yvar, dtype=dtype), name="scale_para")
                shift_para = tf.Variable(tf.constant(ymean, dtype=dtype), name="shift_para")
            wdecode = tf.Variable(tf.random_normal([dim_hidden, dim_input], stddev=1.0 / tf.sqrt(float(dim_hidden)), dtype=dtype), 
                                  name='wdecode')
            xout = tf.matmul(yout, wdecode) * tf.abs(scale_para) + shift_para
        
        with tf.name_scope('loss'):
            monitor = tf.nn.l2_loss(xout - x, name='l2-loss') 
            loss = monitor + alpha * tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=hencode, labels=yout, name='cross_entropy'))\
                    + beta * tf.nn.l2_loss(wdecode, name='regulerization')
            tf.summary.scalar("loss", loss)
            tf.summary.scalar('l2-monitor',monitor)
        
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # optimizer = tf.train.RMSPropOptimizer(learning_rate)
        # optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.minimize(loss)
        
        sess = tf.Session(graph=g)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(tf_board_path+'/train', sess.graph)
        test_writer = tf.summary.FileWriter(tf_board_path+'/test', sess.graph)
        sess.run(tf.global_variables_initializer())
        
        train_err = []
        for i in xrange(max_iter):
            indx = np.random.choice(Y_tr.shape[0], batch_size)
            ybatch = Y_tr[indx].toarray()
            
            if i % 2000 == 0: #record runtime every 2000 step
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE) #trace runtime
                run_metadata = tf.RunMetadata()
                _, monitor_value, loss_value, summary= sess.run([train_op, monitor, loss, merged],
                                                                feed_dict={x: ybatch}, 
                                                                options=run_options,
                                                                run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step{}'.format(i))
                learning_rate = 0.5 * learning_rate
            else: # ordinary training op
                _, monitor_value, loss_value, summary= sess.run([train_op, monitor, loss, merged],
                                                            feed_dict={x: ybatch})
                train_writer.add_summary(summary, i)

            if i % 100 == 0: #validate monitor
                indx = np.random.choice(Y_te.shape[0], batch_size)
                ytest_batch = Y_te[indx].toarray()
                summary = sess.run(merged, 
                                   feed_dict={x:ytest_batch})
                #test_writer.add_run_metadata(run_metadata, 'step{}'.format(i))
                test_writer.add_summary(summary, i)
                

    
        node_list = ['yout', 'pout', 'xout', 'wencode', 'bencode', 'wdecode', 'scale_para', 'shift_para']
        t_vars = tf.trainable_variables()

        para_list = {}
        for var in t_vars:
            para_list[var.name] = sess.run(var)
    
    return g, node_list, para_list, train_err

time.tic()
g, node_list, para_list, train_err = VAE_stoc_neuron(alpha, batch_size, learning_rate, max_iter)
time.toc()

print ('see infomation by running command tensorboard --logir={}'.format(tf_board_path))

In [ ]:
joblib.dump(para_list, model_path+'/paralist.pkl')

In [ ]:
para_list = joblib.load(model_path+'/paralist.pkl')

In [ ]:

# reconstruct the image via the learned codes and templates. It generates the Figure 3. in main text.

W = para_list['encode/wencode:0']
b = para_list['encode/bencode:0']

shift = para_list['decode/scale/shift_para:0']
scale = para_list['decode/scale/scale_para:0']

U = para_list['decode/wdecode:0']

logits = np.dot(Y_tr.toarray(), W) + b
epsilon = 0.5 
pres = 1.0 / (1 + np.exp(-logits))
Z_tr = (np.sign(pres - epsilon) + 1.0) / 2.0

## Step 2: Random Forest CLF and KNN

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=1)
time.tic()
#clf.fit(X_tr, Z_tr)
training_time = time.tocvalue()

In [ ]:
#joblib.dump(clf, os.path.join(model_path , 'label0.pkl'), compress=3)# only one classifiers, name for convention#

In [ ]:
time.tic()
clf = joblib.load(os.path.join(model_path , 'label0.pkl'))
time.toc()

In [ ]:
# faiss brute force search
nn_index = faiss.index_factory(Z_tr.shape[1], "Flat", faiss.METRIC_L2)   # build the index
time.tic()
nn_index.add(Z_tr.astype('float32'))
time.toc()

## Step 3: Predict and Validate

In [ ]:
W = para_list['encode/wencode:0']
b = para_list['encode/bencode:0']

shift = para_list['decode/scale/shift_para:0']
scale = para_list['decode/scale/scale_para:0']

U = para_list['decode/wdecode:0']

Z_te = clf.predict(X_te)

In [ ]:
Y_pred = np.dot(Z_te, U)*scale+shift

In [ ]:
Y_pred_s = scipy.sparse.csr_matrix(Y_pred)

In [ ]:
def precision_at_k(truth, vote, k):
    '''
    evaluate precision at k for a vote vector
    p@k = num of correct prediction in topk / k
    '''
    success = 0
    for i in range(truth.shape[0]):
        # find the k-largest index using partition selet
        # topk are not sorted, np.argsort(vote[topk]) can do that but not needed here
        topk = np.argpartition(vote[i], -k)[-k:] 
        success += np.sum(truth[i, topk])
    return success / ((float(truth.shape[0])*k))

In [ ]:
import util
for i in np.arange(1,6,2):
    print "p@{} for classification:\t {}\n".format(i, precision_at_k(Y_te, Y_pred, i))